# Time Series Test Errors and Cross-Validation

<q>Prediction is very difficult, especially if it's about the future.</q>

--Nils Bohr, Nobel laureate in Physics 

This quote reiterates what we've said about making predictions in the regression material. We can make a model that is very good at fitting the data we have but there is no guarantee that model does well on unseen data (the future in time series).

Thus, before introducing more time series forecasting models we'll quickly review how we can adjust the way we compute test errors and perform cross validation. These techniques allow us to attempt to find a model that predicts well.

## What You'll Accomplish

In particular you'll:
<ul>
    <li>Compute test errors for the average and na&iuml;ve methods on the marathon data,</li>
    <li>Learn about the differences for cross-validation,</li>
    <li>Perform cross-validation on the measles data to compute the best method among the average, na&iuml;ve and, seasonal na&iuml;ve methods</li>
</ul>

Let's import the packages we'll need

In [2]:
# import the packages we'll use
from datetime import datetime

## For data handling
import pandas as pd
import numpy as np

## For plotting
import matplotlib.pyplot as plt
import seaborn as sns

## This sets the plot style
## to have a grid on a white background
sns.set_style("whitegrid")

## Test Errors

Although we are working with time series data, the error measures we use will still be MSE and RMSE. 

In this setting we can refer to the test error as the forecast errors.

Let $T$ be the length of the entire time series, $T_{\text{train}}$ be the length of the training set, let $y_t$ be the target value observed at time $t$, and assume you have some algorithm that produces a prediction, $\hat{y}_t$ at time $t$.

Then the testing MSE is given by:
$$
MSE = \frac{1}{T - T_{\text{train}}} \sum_{t = T_{\text{train}} + 1}^T \left( y_t - \hat{y}_t \right)^2,
$$

and $RMSE = \sqrt{MSE}$.

### Horizons

The <i>horizon</i> of your model is how far out you'd like to predict. For example:
<ul>
    <li>A model that predicts next quarter's earnings has a horizon of $1$ quarter,</li>
    <li>Predicting monthly cases of influenza in 2021 would be a model with a horizon of $1$ year or $12$ months,</li>
    <li>A model predicting daily mileage in July 2020 would have a horizon of $1$ month or $31$ days.</li>
</ul>

Your desired forecasting horizon (how far out into the future you'd like to predict) should determine how large your test set is.

## Practice

Return to the `marathon` data. We'd like to see if we could predict out the winning times over the next $5$ years.

Using both the average method and the na&iuml;ve methods compute the test RMSE. Which method performs better on this data?

In [3]:
marathon = pd.read_csv("marathon.csv", parse_dates = ['year'])

In [7]:
## Make the train test split here.
marathon_copy = marathon.copy()

marathon_train = marathon_copy.iloc[:-5]#last 5 days
marathon_test = marathon_copy.drop(marathon_train.index)

marathon_test

,year,time
115,2012-01-01,132.666667
116,2013-01-01,130.366667
117,2014-01-01,128.616667
118,2015-01-01,129.283333
119,2016-01-01,132.733333


In [11]:
## Calculate the Average Method RMSE here
pred = marathon_train.time.mean()
rmse = np.sqrt(np.sum(np.power(marathon_test.time-pred,2))/len(marathon_test))
rmse

11.93678923328363

In [21]:
## Calculate the Naive Method RMSE here

pred_naive = marathon_train.time[len(marathon_train)-1]
rmse_naive = np.sqrt(np.sum(np.power(marathon_test.time-pred_naive,2))/len(marathon_test))
np.round(rmse_naive,4)

7.8855

## Cross-Validation with Time Series

Just like we can't randomly sample for the train test split, we also can't randomly sample for cross-validation.

Let's explain what you can do with a figure.

<img src="TimeSeriesCV.png" style="width:80%"></img>

Just like we've done up to this point you make a train test split. In the image the training data is blue and the test data is red. In this example our horizon is $4$ time units.

Now using the training data we have made three CV splits. We treat each split like a new data set. In the first one we use the first $4$ time points to predict on the second $4$. Then we use the first eight points to predict on the next $4$. Our final split uses the first $12$ points to predict on the last $4$.

Just like with regression we'd calculate MSE or RMSE on each CV split and find the arithmetic mean across all splits.

If you have a set horizon in mind your cv splits should reflect that horizon. For example, if you use cv to see how well your model predicts $1$ time step forward, but then you actually use that model to predict out $4$ time steps forward you shouldn't be shocked if your predictions don't go too well.

### Practice

Again return to the pre-1963 measles data. Using cross-validation calculate the average RMSE for the average,  na&iuml;ve, and seasonal na&iuml;ve methods. Use a $24$ month horizon. Which model do we expect to perform best?

In [25]:
measles = pd.read_csv("measles.csv", parse_dates=['month'])
measles_pre1963 = measles.loc[measles.month < datetime(1963,1,1),]

In [33]:
## Make the train test split here
measles_copy = measles_pre1963.copy()

measles_train = measles_copy.loc[measles.month < datetime(1961,1,1),]
measles_test = measles_copy.drop(measles_train.index)
measles_test.head()






,month,cases
396,1961-01-01,227
397,1961-02-01,298
398,1961-03-01,374
399,1961-04-01,384
400,1961-05-01,644


In [34]:
## Make CV splits here
test_index = []
train_index = []

for i in range(1,int(len(measles_train)/24)):
    if i > 1:
        test = measles_train.index[-24*i:-24*(i-1)]
    else:
        test = measles_train.index[-24:]
        
    train = measles_train.index[:-24*i]
    
    test_index.append(test)
    train_index.append(train)






In [35]:
## Make any functions you want to calculate
## average rmse here
def get_rmse(pred,actual):
    res = pred - actual
    return np.sqrt(np.sum(np.power(res,2))/len(res))


def naive_forecast(train,test):
    return list(train.cases)[-1]*np.ones(len(test))

def avg_forecast(train,test):
    return train.cases.mean()*np.ones(len(test))

def naive_seasonal_forecast(train,test):
    return list(train.cases)[-len(test):]








In [36]:
## RMSE for Average Method Here 
avg_rmses = []

for i,j in zip(train_index,test_index):
    pred = avg_forecast(measles_train.iloc[i,],measles_train.iloc[j,])
    avg_rmses.append(get_rmse(pred,measles_train.iloc[j,].cases))







In [37]:
## RMSE for the Naive Method here
naive_rmses = []

for i,j in zip(train_index,test_index):
    pred = naive_forecast(measles_train.iloc[i,],measles_train.iloc[j,])
    naive_rmses.append(get_rmse(pred,measles_train.iloc[j,].cases))







In [41]:
## RMSE for the Seasonal Naive method here
seasonal_naive_rmses = []

for i,j in zip(train_index,test_index):
    pred = naive_seasonal_forecast(measles_train.iloc[i,],measles_train.iloc[j,])
    seasonal_naive_rmses.append(get_rmse(pred,measles_train.iloc[j,].cases))
    print(pred)






[823, 789, 949, 1287, 1949, 1840, 845, 335, 176, 170, 231, 832, 1878, 3544, 6410, 7634, 4832, 2594, 893, 204, 86, 58, 47, 92]
[364, 603, 1305, 1853, 1865, 1797, 760, 245, 103, 98, 242, 477, 594, 1242, 2461, 4336, 6064, 4247, 1419, 441, 110, 169, 188, 375]
[224, 246, 334, 518, 645, 600, 522, 215, 164, 209, 318, 1122, 2443, 4876, 10154, 10720, 6136, 4699, 1335, 249, 63, 59, 115, 188]
[282, 381, 549, 1079, 1915, 1770, 948, 380, 184, 246, 515, 1360, 3707, 6441, 8616, 7754, 4945, 2288, 478, 75, 40, 68, 57, 146]
[169, 390, 629, 693, 1008, 960, 454, 234, 133, 100, 214, 483, 1055, 1930, 3776, 4777, 5428, 3440, 1012, 232, 73, 89, 90, 88]
[254, 379, 669, 1141, 1682, 1613, 854, 202, 88, 174, 234, 625, 1709, 3319, 6807, 6909, 5481, 3548, 802, 159, 64, 32, 47, 83]
[60, 82, 171, 207, 221, 358, 194, 68, 18, 79, 181, 231, 809, 2022, 5014, 6597, 4759, 2191, 596, 107, 68, 56, 75, 114]
[232, 877, 2174, 3855, 5724, 5760, 1759, 453, 142, 300, 676, 1675, 2567, 5242, 8498, 5458, 2779, 1151, 182, 63, 21, 26, 

In [39]:
## Compare here
print("Average Method",np.round(np.mean(avg_rmses),4))
print("Naive Method",np.round(np.mean(naive_rmses),4))
print("Seasonal Naive Method",np.round(np.mean(seasonal_naive_rmses),4))







Average Method 2590.5528
Naive Method 2857.4306
Seasonal Naive Method 2500.6819


That's it for this notebook!

Next we'll see one of the two more advanced forecasting techniques touched on in the Time Series notebooks.

### References

Section 3.4 of <a href="https://otexts.com/fpp2/">Forecasting: Principles and Practice</a>, by Rob J Hyndman and George Athanasopoulos.